In [1]:
en_dir="/home/apsisdev/ansary/DATASETS/EnNums/"
bn_dir="/home/apsisdev/ansary/DATASETS/BnNums/"
save_dir="/home/apsisdev/ansary/DATASETS/APSIS/CDR/datasets/numbers/"

In [2]:
import os 
import numpy as np
import cv2
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot  as plt
tqdm.pandas()
import random
from coreLib.processing import correctPadding
from coreLib.utils import create_dir
en_vocab=["0","1","2","3","4","5","6","7","8","9"]
bn_vocab=['০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯']

def random_exec(poplutation=[0,1],weights=[0.5,0.5],match=0):
    return random.choices(population=poplutation,weights=weights,k=1)[0]==match

def get_df_dict(_dir,vocab):
    csv=os.path.join(_dir,"data.csv")
    df=pd.read_csv(csv)
    df.label=df.label.progress_apply(lambda x:str(x))
    df_dict={}
    for v in vocab:
        vdf=df.loc[df.label==v]
        vdf.reset_index(inplace=True,drop=True)
        df_dict[v]=vdf
    return df_dict

def get_rand_img_path(df_dict,comp):
    df=df_dict[comp]
    idx=random.randint(0,len(df)-1)
    return df.iloc[idx,0]

def padAllAround(img,pad_dim,pad_val,pad_single=None):
    '''
        pads all around the image
    '''
    if pad_single is None:
        h,w=img.shape
        # pads
        left_pad =np.ones((h,pad_dim))*pad_val
        right_pad=np.ones((h,pad_dim))*pad_val
        # pad
        img =np.concatenate([left_pad,img,right_pad],axis=1)
        # shape
        h,w=img.shape
        top_pad =np.ones((pad_dim,w))*pad_val
        bot_pad=np.ones((pad_dim,w))*pad_val
        # pad
        img =np.concatenate([top_pad,img,bot_pad],axis=0)
    elif pad_single=="tb":
        # shape
        h,w=img.shape
        top_pad =np.ones((pad_dim,w))*pad_val
        bot_pad=np.ones((pad_dim,w))*pad_val
        # pad
        img =np.concatenate([top_pad,img,bot_pad],axis=0)
    else:
        h,w=img.shape
        # pads
        left_pad =np.ones((h,pad_dim))*pad_val
        right_pad=np.ones((h,pad_dim))*pad_val
        # pad
        img =np.concatenate([left_pad,img,right_pad],axis=1)
    return img

def create_labeled_box(num_box):
    labeled=[]
    bsize=random.randint(64,96)
    side=random.randint(0,10)
    thickness=random.randint(2,8)
    pad=random.randint(2,5)
    for i in range(num_box):
        iden=i+2
        box=np.ones((bsize,bsize))*iden
        box=padAllAround(box,thickness,0)
        box=padAllAround(box,pad,1)
        labeled.append(box)
    box=np.concatenate(labeled,axis=-1)
    if random_exec():
        box=padAllAround(box,side,1,pad_single="tb")
    return  box.astype("uint8")

def randColor(col=True):
    '''
        generates random color
    '''
    if col:
        return (random.randint(0,255),random.randint(0,255),random.randint(0,255))
    else:
        d=random.randint(0,64)
        return (d,d,d)

def noisy(image):
    row,col,ch= image.shape
    mean = 0
    var = random.randint(1,2)/10
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    
    return noisy


def create_number_image(df_dict,data):
    cimgs=[cv2.imread(get_rand_img_path(df_dict,c)) for c in data]
    box_map=create_labeled_box(len(data))
    midxs=sorted(np.unique(box_map))[2:]

    h,w=box_map.shape
    mask=np.zeros_like(box_map) 
    img=np.ones((h,w,3))*255
    
    back=np.copy(img)
    back[box_map==0]=randColor()
    back=cv2.GaussianBlur(back,(5,5),0) 
    img=noisy(img)
    back=noisy(back)
    
    for cimg,midx in zip(cimgs,midxs):
        # placement
        idx = np.where(box_map==midx)                
        x_min,x_max = np.min(idx[1]), np.max(idx[1])
        y_min,y_max=0,h
        cw=x_max-x_min
        # fix
        cimg=cv2.resize(cimg,(cw,h))
        img[y_min:y_max,x_min:x_max]=cimg

    img[box_map==0]=back[box_map==0]
    img=img.astype("uint8")
    return img

In [3]:
# data="01786044716"
# df_dict=get_df_dict(en_dir,en_vocab)
# plt.imshow(create_number_image(df_dict,data))

In [5]:
num_samples=1000000
_vocab=en_vocab
_dir=en_dir
lang_dir=create_dir(save_dir,"english")
img_dir=create_dir(lang_dir,"images")
data_csv=os.path.join(lang_dir,"data.csv")
df_dict=get_df_dict(_dir,_vocab)
words=[]
labels=[]
masks=[]
filepaths=[]
max_len=15

for i in tqdm(range(num_samples)):
    try:
        if random_exec(weights=[0.65,0.35]):
            dlen=11
        else:
            if random_exec(weights=[0.65,0.35]):
                dlen=2
            else:
                dlen=random.randint(3,10)
        label="".join([random.choice(_vocab) for _ in range(dlen)])
        #d-w
        words.append(label)

        img=create_number_image(df_dict,label)
        img,mask=correctPadding(img,dim=(64,512),ptype="left")
        #d-m
        masks.append(mask)
        # d-f
        filepath=os.path.join(img_dir,f"{i}.png")
        cv2.imwrite(filepath,img)
        filepaths.append(filepath)
        # d-l
        label=[_vocab.index(c)+1 for c in label]
        for _ in range(max_len-len(label)):
            label.append(0)
        labels.append(label)
    except Exception as e:
        print(e)


df=pd.DataFrame({"filepath":filepaths,"label":labels,"word":words,"mask":masks})
df.to_csv(data_csv,index=False)
df

100%|██████████| 70000/70000 [00:00<00:00, 1519982.19it/s]


  0%|          | 0/1000000 [00:00<?, ?it/s]

,filepath,label,word,mask
0,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[6, 2, 5, 10, 4, 4, 5, 9, 4, 8, 8, 0, 0, 0, 0]",51493348377,512
1,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",55,128
2,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[9, 9, 7, 8, 8, 6, 5, 10, 2, 9, 5, 0, 0, 0, 0]",88677549184,512
3,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[3, 6, 2, 7, 1, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0]",251609,384
4,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[4, 5, 3, 4, 3, 7, 8, 2, 2, 5, 7, 0, 0, 0, 0]",34232671146,512
...,...,...,...,...
999995,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[10, 4, 5, 1, 4, 10, 5, 6, 2, 6, 6, 0, 0, 0, 0]",93403945155,512
999996,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[8, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",70,128
999997,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[8, 9, 8, 6, 9, 9, 7, 4, 7, 1, 1, 0, 0, 0, 0]",78758863600,512
999998,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[7, 4, 10, 3, 8, 3, 5, 9, 0, 0, 0, 0, 0, 0, 0]",63927248,459


In [6]:
num_samples=1000000
_vocab=bn_vocab
_dir=bn_dir
lang_dir=create_dir(save_dir,"bangla")
img_dir=create_dir(lang_dir,"images")
data_csv=os.path.join(lang_dir,"data.csv")
df_dict=get_df_dict(_dir,_vocab)
words=[]
labels=[]
masks=[]
filepaths=[]
max_len=15

for i in tqdm(range(num_samples)):
    try:
        if random_exec(weights=[0.65,0.35]):
            dlen=11
        else:
            if random_exec(weights=[0.65,0.35]):
                dlen=2
            else:
                dlen=random.randint(3,10)
        label="".join([random.choice(_vocab) for _ in range(dlen)])
        #d-w
        words.append(label)

        img=create_number_image(df_dict,label)
        img,mask=correctPadding(img,dim=(64,512),ptype="left")
        #d-m
        masks.append(mask)
        # d-f
        filepath=os.path.join(img_dir,f"{i}.png")
        cv2.imwrite(filepath,img)
        filepaths.append(filepath)
        # d-l
        label=[_vocab.index(c)+1 for c in label]
        for _ in range(max_len-len(label)):
            label.append(0)
        labels.append(label)
    except Exception as e:
        print(e)


df=pd.DataFrame({"filepath":filepaths,"label":labels,"word":words,"mask":masks})
df.to_csv(data_csv,index=False)
df

100%|██████████| 102875/102875 [00:00<00:00, 1647338.51it/s]


  0%|          | 0/1000000 [00:00<?, ?it/s]

,filepath,label,word,mask
0,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[9, 6, 1, 8, 7, 10, 4, 4, 2, 8, 2, 0, 0, 0, 0]",৮৫০৭৬৯৩৩১৭১,512
1,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[4, 2, 4, 8, 4, 4, 9, 10, 3, 8, 10, 0, 0, 0, 0]",৩১৩৭৩৩৮৯২৭৯,512
2,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[4, 5, 10, 2, 6, 9, 10, 7, 8, 6, 10, 0, 0, 0, 0]",৩৪৯১৫৮৯৬৭৫৯,512
3,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[3, 5, 10, 2, 9, 9, 2, 10, 1, 3, 7, 0, 0, 0, 0]",২৪৯১৮৮১৯০২৬,512
4,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[1, 2, 10, 3, 7, 4, 1, 5, 10, 1, 4, 0, 0, 0, 0]",০১৯২৬৩০৪৯০৩,512
...,...,...,...,...
999995,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[9, 2, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",৮১৭,188
999996,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[7, 4, 6, 9, 1, 10, 9, 9, 7, 5, 4, 0, 0, 0, 0]",৬৩৫৮০৯৮৮৬৪৩,512
999997,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",৯৩,110
999998,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[3, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",২৭,111


In [14]:

import os 
import json
import math
import pandas as pd 
import tensorflow as tf
import numpy as np 
from ast import literal_eval
from tqdm.auto import tqdm
tqdm.pandas()
#---------------------------------------------------------------
# data functions
#---------------------------------------------------------------
cols=["filepath","label","word","mask"]
eval_cols=["label"]
    
# feature fuctions
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _int64_list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def toTfrecord(df,rnum,rec_path):
    '''
        args:
            df      :   the dataframe that contains the information to store
            rnum    :   record number
            rec_path:   save_path
            mask_dim:   the dimension of the mask
    '''
    tfrecord_name=f'{rnum}.tfrecord'
    tfrecord_path=os.path.join(rec_path,tfrecord_name) 
    with tf.io.TFRecordWriter(tfrecord_path) as writer:    
        
        for idx in range(len(df)):
            # base
            img_path=df.iloc[idx,0]
            # img
            with(open(img_path,'rb')) as fid:
                image_png_bytes=fid.read()
            # feature desc
            data ={ 'image':_bytes_feature(image_png_bytes)}

            for cidx,col in enumerate(cols):
                if col in eval_cols:
                    data[col]=_int64_list_feature(df.iloc[idx,cidx]) 

            
            features=tf.train.Features(feature=data)
            example= tf.train.Example(features=features)
            serialized=example.SerializeToString()
            writer.write(serialized)  

def createRecords(data,save_path,tf_size=10240):
    '''
        creates tf records:
        args:
            data        :   either the csv path or a dataframe
            save_path   :   location to save tfrecords
    '''
    if type(data)==str:
        data=pd.read_csv(data)
        for col in eval_cols:
            data[col]=data[col].progress_apply(lambda x: literal_eval(x))
    
    for idx in tqdm(range(0,len(data),tf_size)):
        df        =   data.iloc[idx:idx+tf_size]  
        rnum      =   idx//tf_size
        toTfrecord(df,rnum,save_path)

    
    

In [15]:
lang_dir=create_dir(save_dir,"english")
rec_dir=create_dir(lang_dir,"tfrecords")
data=os.path.join(lang_dir,"data.csv")
createRecords(data,rec_dir)

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

In [17]:
lang_dir=create_dir(save_dir,"bangla")
rec_dir=create_dir(lang_dir,"tfrecords")
data=os.path.join(lang_dir,"data.csv")
createRecords(data,rec_dir)

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

In [18]:
# formulate robust scanner as backup

In [22]:
# classification
bn=os.path.join(save_dir,"bangla","data.csv")
en=os.path.join(save_dir,"english","data.csv")
bn=pd.read_csv(bn)
en=pd.read_csv(en)
bn=bn.sample(frac=1)
bn.reset_index(inplace=True,drop=True)
en=en.sample(frac=1)
en.reset_index(inplace=True,drop=True)
bn=bn[:100000]
en=en[:100000]
df=pd.concat([bn,en],ignore_index=True)
df=df.sample(frac=1)
df.reset_index(inplace=True,drop=True)
df

,filepath,label,word,mask
0,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[8, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",75,118
1,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[4, 3, 7, 5, 2, 1, 9, 10, 4, 10, 2, 0, 0, 0, 0]",32641089391,512
2,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[2, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",১৪,128
3,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[5, 7, 8, 9, 4, 9, 5, 1, 4, 7, 10, 0, 0, 0, 0]",46783840369,512
4,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[7, 6, 8, 8, 1, 9, 4, 1, 1, 4, 6, 0, 0, 0, 0]",65770830035,512
...,...,...,...,...
199995,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[8, 9, 2, 4, 3, 6, 8, 4, 5, 8, 2, 0, 0, 0, 0]",78132573471,512
199996,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",32,128
199997,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[9, 3, 3, 9, 9, 1, 4, 9, 9, 2, 8, 0, 0, 0, 0]",৮২২৮৮০৩৮৮১৭,512
199998,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[9, 1, 2, 9, 10, 3, 7, 2, 10, 2, 8, 0, 0, 0, 0]",80189261917,512


In [23]:
df.label=df.filepath.progress_apply(lambda x: [1,0] if "bangla" in x else [0,1])
df

  0%|          | 0/200000 [00:00<?, ?it/s]

,filepath,label,word,mask
0,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[0, 1]",75,118
1,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[0, 1]",32641089391,512
2,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[1, 0]",১৪,128
3,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[0, 1]",46783840369,512
4,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[0, 1]",65770830035,512
...,...,...,...,...
199995,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[0, 1]",78132573471,512
199996,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[0, 1]",32,128
199997,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[1, 0]",৮২২৮৮০৩৮৮১৭,512
199998,/home/apsisdev/ansary/DATASETS/APSIS/CDR/datas...,"[0, 1]",80189261917,512


In [24]:
rec_dir=create_dir(save_dir,"classification")
createRecords(df,rec_dir)

  0%|          | 0/20 [00:00<?, ?it/s]